# Data Science Regression Project: Predicting Apartment Prices in Germany


In [43]:
from os import listdir
import pandas as pd
import re

In [44]:
# finding json files in givin path 
def find_csv_filenames(path_to_dir, suffix=".csv" ):
    filenames = listdir(path_to_dir)
    return [filename for filename in filenames if filename.endswith(suffix)]

In [45]:
def concatenate_path_csv_file():
    path_with_csv_files = []
    for csv_file in csv_files:
        path_with_csv_files.append(path+csv_file)
    return path_with_csv_files

## Data Load: Load germany apartment data, to see features


In [46]:
path = r"./house_data/"
csv_files = find_csv_filenames(path)
df = pd.concat(map(pd.read_csv, concatenate_path_csv_file()), ignore_index=True)

In [47]:
df.head(5)

,sales_price,livable_surface,land,number_of_rooms,location
0,"548,000€",123m²,894m²,3,-Merzkirchen
1,"64,000€",76m²,612m²,4,Danneilweg-Salzwedel
2,"485,000€",180m²,"71,600m²",6,"ImHainbruch16,54317Gusterath-Gusterath"
3,"47,000€",96m²,117m²,4,IllingerStraße-Heusweiler
4,"595,000€",389m²,"1,230m²",15,-Zerf


In [48]:
# we have 1116 houses and 5 features
df.shape

(1116, 5)

In [49]:
# sales_price is the dependent feature, the rest are independet. We try to predict the sales_price
df.columns

Index(['sales_price', 'livable_surface', 'land', 'number_of_rooms',
       'location'],
      dtype='object')

In [50]:
# unique method is used to get an overview of unique elements in specific index.
df['location'].unique()

array(['-Merzkirchen', 'Danneilweg-Salzwedel',
       'ImHainbruch16,54317Gusterath-Gusterath',
       'IllingerStraße-Heusweiler', '-Zerf', 'ZumWaldhaus-Riedenburg',
       'HöwischerStraße-Arendsee', 'Ernteweg-Kulmbach', '-Rittersdorf',
       '-Retterath', '-Trier', '-Wadern', '-Perl', '-Saarbrücken',
       'Riedhofstraße-Weißenhorn', 'NadorsterStraße-Oldenburg',
       '-Minderlittgen', '-Hentern', '-Mettlach', '-Alsdorf',
       '-Wincheringen', 'Hindenburgstraße-Herzberg', '-Langsur',
       'Kirchenstrasse-Perl', '-Bollendorf', '-Kaiserslautern',
       '-Neuerburg', 'Dorfstraße-Gera', '-Speicher',
       'MerzigTheexactadressisnotindicated',
       'PerlTheexactadressisnotindicated', '-Waxweiler', '-Großrosseln',
       '-Losheim', '-Übereisenbach', 'Herzogstraße-Duisburg', '-Malberg',
       'Hauptsrasse-Kleinblittersdorf', '-Saarlouis', '-Taben-Rodt',
       '-Riveris', '-Waldweiler', 'Grasberg-BadNeustadt', '-Saarburg',
       '-Schillingen', '-Waldrach', 'AmDreschplatz-Lin

In [51]:
# removing streets name from location
df['location'] = df['location'].apply(lambda x: x.split('-')[-1])

In [52]:
df.head()

,sales_price,livable_surface,land,number_of_rooms,location
0,"548,000€",123m²,894m²,3,Merzkirchen
1,"64,000€",76m²,612m²,4,Salzwedel
2,"485,000€",180m²,"71,600m²",6,Gusterath
3,"47,000€",96m²,117m²,4,Heusweiler
4,"595,000€",389m²,"1,230m²",15,Zerf


In [53]:
# first location is the most frequently-occurring location. 
df['location'].value_counts()

Wadern         22
Saarbrücken    21
Trier          16
Perl           12
Losheim        10
               ..
Noviand         1
Frielendorf     1
Heemsen         1
Mühlhausen      1
Altenbeken      1
Name: location, Length: 685, dtype: int64

## Drop features that are not required to build our model

In [54]:
# we will work also with location to show how to handle categorical data
df_draft = df.drop(['location'],axis='columns')
df_draft.shape

(1116, 4)

## Handle missing data if its NaN. 

In [55]:
'''
We have no NaN values but something like: sales_price: Priceonrequest
'''
df = df.dropna()
df.isnull().sum()

sales_price        0
livable_surface    0
land               0
number_of_rooms    0
location           0
dtype: int64

In [58]:
index_sales_price = df.columns.get_loc('sales_price')
price_pattern = r'([0-9]*(,|.)?[0-9]*(,|.)?[0-9]*\€)'

for row in range(0, len(df)-1):
    try:
        price = re.search(price_pattern, df.iat[row, index_sales_price]).group()
    except Exception as e:
        # drops sales_prices values which does not contains numerical value e.g. Priceonrequest
        df = df.drop(df.index[[row]])
    price = 'None'

In [59]:
df.head()

,sales_price,livable_surface,land,number_of_rooms,location
0,"548,000€",123m²,894m²,3,Merzkirchen
1,"64,000€",76m²,612m²,4,Salzwedel
2,"485,000€",180m²,"71,600m²",6,Gusterath
3,"47,000€",96m²,117m²,4,Heusweiler
4,"595,000€",389m²,"1,230m²",15,Zerf


In [60]:
# remove € sign and ',' from entire sales_prices...
df['sales_price'] = df['sales_price'].replace({r'\€':''}, regex = True)
df['sales_price'] = df['sales_price'].replace({r',':'.'}, regex = True)
df['livable_surface'] = df['livable_surface'].replace({r'm\u00b2':''}, regex = True)
df['land'] = df['land'].replace({r'm\u00b2':''}, regex = True)
df.rename(columns={'sales_price': 'sales_price', 'livable_surface': 'livable_surface_in_m\u00b2','land':'land_in_m\u00b2', 'number_of_rooms': 'number_of_rooms', 'location':'location'}, inplace=True)

In [61]:
df.head()

,sales_price,livable_surface_in_m²,land_in_m²,number_of_rooms,location
0,548.000,123,894,3,Merzkirchen
1,64.000,76,612,4,Salzwedel
2,485.000,180,"71,600",6,Gusterath
3,47.000,96,117,4,Heusweiler
4,595.000,389,"1,230",15,Zerf


In [62]:
# store dataframe into csv
df.to_csv (f'{path}export_dataframe.csv',index = False, header=True)